# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install -U adapter-transformers
!pip install datasets

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [9]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("ag_news")
dataset.num_rows

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

{'test': 7600, 'train': 120000}

Every dataset sample has an input text and a binary label:

In [10]:
max([len(x['text']) for x in dataset['train']])

1012

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [12]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-1c11d29ee2e50e78.arrow


  0%|          | 0/8 [00:00<?, ?ba/s]

Now we're ready to train our model...

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'text'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'text'],
        num_rows: 7600
    })
})

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [14]:
import numpy as np

In [15]:
nLabels = len(np.unique(dataset['train']['labels']))
nLabels

4

In [16]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=nLabels,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [19]:
adapter_name = model.load_adapter("/content/drive/MyDrive/swp/AGnews Results/final_adapter")

Overwriting existing adapter 'ag_news'.
Overwriting existing head 'ag_news'


In [ ]:
# # Add a new adapter
# model.add_adapter("sciie")
# # Add a matching classification head
# model.add_classification_head(
#     "sciie",
#     num_labels=7,
#     # id2label={ 0: "👎", 1: "👍"}
#   )

# # Activate the adapter
# model.train_adapter("sciie")

In [20]:
model.set_active_adapters(adapter_name)

In [21]:
import torch
device='cuda'

In [22]:
# test_dataset[0]

In [25]:
test_dataset = dataset['test']
bsz = 16
# bsz = test_dataset.num_rows
i = 0
batches = []
while i<test_dataset.num_rows:
  batches.append(test_dataset[i:i+bsz])
  i+=bsz

  

In [27]:
# Put model in evaluation mode
model.to(device)
model.eval()


# Tracking variables for storing ground truth and predictions 
predictions , true_labels = [], []

# Prediction Loop
for i,batch in enumerate(batches):
  print(i)

 
 
  # Unpack the inputs from our dataloader and move to GPU/accelerator 
 
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(input_ids, attention_mask=attention_mask, 
                         labels=labels)

  logits = outputs[1]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [28]:
# import matplotlib.pyplot as plt

In [29]:
# fig, a = plt.subplots(2,3)
# a[0,0].hist(logits[:,0])
# a[0,1].hist(logits[:,1])
# a[0,2].hist(logits[:,2])
# a[1,0].hist(logits[:,3])
# a[1,1].hist(logits[:,4])
# a[1,2].hist(logits[:,5])

In [42]:
labels = np.array(true_labels).flatten()
prds = np.array(predictions).reshape(475*16,4).argmax(axis=1)

In [44]:
from sklearn.metrics import f1_score, classification_report

In [45]:
f1_score(labels, prds, average='micro')

0.9500000000000001

In [46]:
print(classification_report(labels, prds, labels=list(range(nLabels)), digits=4))

              precision    recall  f1-score   support

           0     0.9676    0.9574    0.9624      1900
           1     0.9889    0.9884    0.9887      1900
           2     0.9223    0.9189    0.9206      1900
           3     0.9217    0.9353    0.9284      1900

    accuracy                         0.9500      7600
   macro avg     0.9501    0.9500    0.9500      7600
weighted avg     0.9501    0.9500    0.9500      7600



In [ ]:
sum(dataset['train']['labels']==5)

tensor(60)

In [ ]:
# from datasets import list_metrics
# metrics_list = list_metrics()
# # len(metrics_list)

# print(', '.join(metric for metric in metrics_list))

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [ ]:
# import torch, gc

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# import numpy as np
# from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

# training_args = TrainingArguments(
#     learning_rate=2*1e-5,
#     num_train_epochs=30,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     logging_steps=200,
#     output_dir="./training_output",
#     overwrite_output_dir=True,
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
# )

# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}

# trainer = AdapterTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     compute_metrics=compute_accuracy,
# )

Start the training 🚀

In [ ]:
# trainer.train()

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [ ]:
# trainer.evaluate()

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

In [ ]:
# from transformers import TextClassificationPipeline

# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

# classifier("This is awesome!")

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [ ]:
# model.save_adapter("./final_adapter", "sciie")

# !ls -lh final_adapter

**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.

In [ ]:
# from google.colab import files
# !zip -r /content/file.zip /content
# files.download("/content/file.zip")